In [33]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [34]:
from models.modeling_xvla import XVLA
from models.processing_xvla import XVLAProcessor

model = XVLA.from_pretrained("2toINF/X-VLA-SoftFold", trust_remote_code=True)
processor = XVLAProcessor.from_pretrained("2toINF/X-VLA-SoftFold", trust_remote_code=True)

In [35]:
import torch
model = model.to(device="cuda", dtype=torch.bfloat16)

In [36]:
import torch
from datasets.domain_config import DATA_WEIGHTS, DATA_DOMAIN_ID

# TODO
model.train(mode=False)

input_ids = processor.encode_language("do something")["input_ids"]
image_input = torch.full((1, 3, 3, 224, 224), fill_value=0.)
image_mask = torch.full((1, 3), fill_value=True)
domain_id = torch.full((1,), fill_value=DATA_DOMAIN_ID["lift2"])
proprio = torch.full((1, 20), fill_value=0.)
action = torch.full((1, 30, 20), fill_value=0.)


In [37]:
generate_actions = torch.compile(
    model.generate_actions, 
    fullgraph=True, 
    dynamic=False, 
    mode="max-autotune",
)

In [38]:
import torch
torch.set_float32_matmul_precision("high")

In [39]:
generate_actions(
    input_ids=input_ids.to(model.device, non_blocking=True),
    image_input=image_input.to(model.device, non_blocking=True, dtype=torch.bfloat16),
    image_mask=image_mask.to(model.device, non_blocking=True),
    domain_id=domain_id.to(model.device, non_blocking=True),
    proprio=proprio.to(model.device, non_blocking=True, dtype=torch.bfloat16),
)

tensor([[[-3.3594e-01, -2.5195e-01, -2.4902e-01, -2.4219e-01,  4.1992e-01,
           1.4648e-01,  4.5312e-01,  1.1670e-01, -2.7734e-01,  4.1797e-01,
           3.1250e-01,  4.2383e-01, -4.1992e-01, -2.3047e-01, -2.2070e-01,
           1.0840e-01, -3.7109e-01, -1.2939e-02, -4.0430e-01,  6.0547e-01],
         [-3.3203e-01, -2.4609e-01, -2.5195e-01, -2.4219e-01,  4.2969e-01,
           1.4648e-01,  4.5117e-01,  1.1914e-01, -2.6953e-01,  4.1602e-01,
           3.0273e-01,  4.3945e-01, -4.2188e-01, -2.2559e-01, -2.2168e-01,
           1.1279e-01, -3.6328e-01, -1.0803e-02, -3.9844e-01,  6.0547e-01],
         [-3.3398e-01, -2.3438e-01, -2.5781e-01, -2.3340e-01,  4.4141e-01,
           1.4746e-01,  4.4727e-01,  1.2354e-01, -2.5977e-01,  4.1406e-01,
           2.8906e-01,  4.4922e-01, -4.3164e-01, -2.1875e-01, -2.2070e-01,
           1.0449e-01, -3.5938e-01, -1.1597e-03, -3.9648e-01,  6.0547e-01],
         [-3.3789e-01, -2.3047e-01, -2.6562e-01, -2.3145e-01,  4.4336e-01,
           1.4551e-01,

In [41]:
%%timeit -n 10

generate_actions(
    input_ids=input_ids.to(model.device, non_blocking=True),
    image_input=image_input.to(model.device, non_blocking=True, dtype=torch.bfloat16),
    image_mask=image_mask.to(model.device, non_blocking=True),
    domain_id=domain_id.to(model.device, non_blocking=True),
    proprio=proprio.to(model.device, non_blocking=True, dtype=torch.bfloat16),
)

36 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
with torch.profiler.profile() as prof:
    generate_actions(
        input_ids=input_ids.to(model.device, non_blocking=True),
        image_input=image_input.to(model.device, non_blocking=True, dtype=torch.bfloat16),
        image_mask=image_mask.to(model.device, non_blocking=True),
        domain_id=domain_id.to(model.device, non_blocking=True),
        proprio=proprio.to(model.device, non_blocking=True, dtype=torch.bfloat16),
    )
prof.export_chrome_trace("trace.json")

In [31]:
actions = generate_actions(
    input_ids=input_ids.to(model.device, non_blocking=True),
    image_input=image_input.to(model.device, non_blocking=True, dtype=torch.bfloat16),
    image_mask=image_mask.to(model.device, non_blocking=True),
    domain_id=domain_id.to(model.device, non_blocking=True),
    proprio=proprio.to(model.device, non_blocking=True, dtype=torch.bfloat16),
)

In [32]:
%%timeit -n 10

import io
import numpy

buffer = io.BytesIO()

numpy.save(
    buffer,
    arr=actions.to(torch.float32).numpy(force=True),
    allow_pickle=False,
)


The slowest run took 8.19 times longer than the fastest. This could mean that an intermediate result is being cached.
1.15 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
